<a href="https://colab.research.google.com/github/jeaneigsi/Humain-/blob/main/Flux_trainer_khoya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone -b sd3 https://github.com/kohya-ss/sd-scripts.git
%cd sd-scripts
!pip install torch==2.4.0 torchvision==0.19.0 --index-url https://download.pytorch.org/whl/cu124
!pip install -r requirements.txt

In [2]:
!mkdir -p ./sd3/  && wget -o  ./sd3/clip_l.safetensors  https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors
!wget https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev.sft
!wget -o  .sd3/t5xxl_fp16.safetensors  https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp16.safetensors
!wget https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft

--2024-08-26 13:04:59--  https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev.sft
Resolving huggingface.co (huggingface.co)... 13.35.7.57, 13.35.7.38, 13.35.7.81, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.57|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/2d/6c/2d6cde08f0ddbdceafbcce501fdc08ef2283be0fcd41e4159060967c30c68d8f/4610115bb0c89560703c892c59ac2742fa821e60ef5871b33493ba544683abd7?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27flux1-dev.sft%3B+filename%3D%22flux1-dev.sft%22%3B&Expires=1724936699&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNDkzNjY5OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzJkLzZjLzJkNmNkZTA4ZjBkZGJkY2VhZmJjY2U1MDFmZGMwOGVmMjI4M2JlMGZjZDQxZTQxNTkwNjA5NjdjMzBjNjhkOGYvNDYxMDExNWJiMGM4OTU2MDcwM2M4OTJjNTlhYzI3NDJmYTgyMWU2MGVmNTg3MWIzMzQ5M2JhNTQ0NjgzYWJkNz9yZXNwb25zZS1jb2

In [3]:
!mkdir -p ./dataset_1024_bs2.toml

In [ ]:
!accelerate launch --mixed_precision bf16 --num_cpu_threads_per_process 1 flux_train_network.py --pretrained_model_name_or_path flux1-dev.sft --clip_l sd3/clip_l.safetensors --t5xxl sd3/t5xxl_fp16.safetensors --ae ae.sft --cache_latents_to_disk --save_model_as safetensors --sdpa --persistent_data_loader_workers --max_data_loader_n_workers 2 --seed 42 --gradient_checkpointing --mixed_precision bf16 --save_precision bf16 --network_module networks.lora_flux --network_dim 4 --optimizer_type adafactor --optimizer_args "relative_step=False" "scale_parameter=False" "warmup_init=False" --network_train_unet_only --cache_text_encoder_outputs --cache_text_encoder_outputs_to_disk --fp8_base --highvram --max_train_epochs 4 --save_every_n_epochs 1 --dataset_config dataset_1024_bs2.toml --output_dir ./ --output_name flux-lora-name --timestep_sampling sigmoid --model_prediction_type raw --guidance_scale 1.0 --loss_type l2 --split_mode --network_args "train_blocks=single"
